## Detectando placas com modelo Yolo treinado

In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/107.7 GB disk)


In [7]:
import cv2
from ultralytics import YOLO
import os

# Carrega o modelo YOLO
model = YOLO('/content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/detectando_placa_yolo/modelo_yolo_treinado/best.pt')

# Função para extrair a região da placa
def extract_plate(results):
    detections = results[0]
    for detection in detections:
        if detection.names[0] == 'placa':
            x_min, y_min, x_max, y_max = map(int, detection.boxes[0].xyxy[0].tolist())
            return (x_min, y_min, x_max, y_max)
    return None

# Função de detecção e salvamento
def detect_and_save(image_path, save_dir):
    # Extrai o nome do arquivo da imagem
    arquivo = os.path.basename(image_path).split('.')[0]

    # Cria uma pasta com o nome da imagem
    pasta = os.path.join(save_dir, arquivo)
    os.makedirs(pasta, exist_ok=True)

    # Carrega a imagem
    img = cv2.imread(image_path)

    # Salva a imagem original na pasta
    original_image_save_path = os.path.join(pasta, f'{arquivo}.jpg')
    cv2.imwrite(original_image_save_path, img)

    # Executa a detecção com o modelo YOLO
    results = model.predict(source=image_path)

    # Extrai e salva a placa se for detectada
    plate_coords = extract_plate(results)
    if plate_coords:
        x_min, y_min, x_max, y_max = plate_coords
        plate_img = img[y_min:y_max, x_min:x_max]
        plate_image_save_path = os.path.join(pasta, f'{arquivo}_placa.jpg')
        cv2.imwrite(plate_image_save_path, plate_img)
        print(f"Placa detectada e salva em {plate_image_save_path}")
    else:
        print("Nenhuma placa detectada.")

    # Salva a imagem anotada com as detecções
    for result in results:
        annotated_frame = result.plot()
        annotated_image_save_path = os.path.join(pasta, f'{arquivo}_detec.jpg')
        cv2.imwrite(annotated_image_save_path, annotated_frame)

        # Verifica se a imagem foi salva corretamente
        if os.path.exists(annotated_image_save_path):
            print(f"Imagem anotada salva com sucesso em {annotated_image_save_path}")
        else:
            print(f"Erro ao salvar a imagem anotada em {annotated_image_save_path}")

# Função para processar todas as imagens em uma pasta
def processar_imagens_na_pasta(pasta, save_dir):
    for arquivo in os.listdir(pasta):
        if arquivo.endswith(".jpg") or arquivo.endswith(".png"):
            print(f"Processando: {arquivo}")
            caminho_completo = os.path.join(pasta, arquivo)
            detect_and_save(caminho_completo, save_dir)

input_folder = '/content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/placas/images'
output_folder = '/content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/detectando_placa_yolo'

# Processa todas as imagens na pasta de entrada
processar_imagens_na_pasta(input_folder, output_folder)

Processando: img_000026.jpg

image 1/1 /content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/placas/images/img_000026.jpg: 384x640 1 placa, 4363.4ms
Speed: 13.1ms preprocess, 4363.4ms inference, 22.2ms postprocess per image at shape (1, 3, 384, 640)
Placa detectada e salva em /content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/detectando_placa_yolo/img_000026/img_000026_placa.jpg
Imagem anotada salva com sucesso em /content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/detectando_placa_yolo/img_000026/img_000026_detec.jpg
Processando: img_003374.jpg

image 1/1 /content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/placas/images/img_003374.jpg: 384x640 1 placa, 3041.4ms
Speed: 2.6ms preprocess, 3041.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Placa detectada e salva em /content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/detectando_placa_yolo/img_003374/img_003374_placa.jp

## Métricas

In [10]:
# Função para contar todas as imagens detectadas (placa.jpg) em uma pasta e suas subpastas
def conta_imagens_detectadas_recursivo(pasta_raiz):
    quant_placas = 0
    for root, dirs, files in os.walk(pasta_raiz):
        for arquivo in files:
            if arquivo.endswith("placa.jpg"):
                quant_placas += 1
    return quant_placas
input_folder = '/content/drive/MyDrive/ColabNotebooks/tcc_mba/validacao_modelo_yolo_ocr/detectando_placa_yolo'

# Processa todas as imagens em todas as subpastas
quantidade = conta_imagens_detectadas_recursivo(input_folder)
print(f"Quantidade de placas detectadas: {quantidade}")

Quantidade de placas detectadas: 500


## Das 500 placas que foi proposto para o modelo identificar o modelo identificou as 500.